<a href="https://colab.research.google.com/github/premkumar6/Deep_learning/blob/main/Evalutation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

MNIST MODEL

In [2]:
inputs = keras.Input(shape=(784,),name="digits")
x1 = layers.Dense(64,activation="relu")(inputs)
x2 = layers.Dense(64, activation="relu")(x1)
outputs = layers.Dense(10,name="predictions")(x2)
model = keras.Model(inputs=inputs,outputs=outputs)

In [7]:
#Optimizer
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
#Loss_fn
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#preparing training dataset
batch_size=64
(x_train,y_train), (x_test,y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train,(-1,784))
x_test = np.reshape(x_test,(-1,784))

x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)


In [8]:
epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_batch_train, logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * batch_size))



Start of epoch 0
Training loss (for one batch) at step 0: 110.7861
Seen so far: 64 samples
Training loss (for one batch) at step 200: 1.0152
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.5612
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.8685
Seen so far: 38464 samples

Start of epoch 1
Training loss (for one batch) at step 0: 1.0548
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.7783
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.5878
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.4785
Seen so far: 38464 samples


In [9]:
# Get model
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [10]:
import time

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric.update_state(y_batch_train, logits)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))



Start of epoch 0
Training loss (for one batch) at step 0: 62.2942
Seen so far: 64 samples
Training loss (for one batch) at step 200: 1.4107
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 1.4122
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 1.3910
Seen so far: 38464 samples
Training acc over epoch: 0.7864
Validation acc: 0.8067
Time taken: 10.66s

Start of epoch 1
Training loss (for one batch) at step 0: 1.2072
Seen so far: 64 samples
Training loss (for one batch) at step 200: 1.0819
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.6997
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.2722
Seen so far: 38464 samples
Training acc over epoch: 0.8615
Validation acc: 0.8608
Time taken: 10.66s


Adding Tf.function, Training Modul runs faster

In [14]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

In [15]:
@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

In [16]:
import time

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 0.6774
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.7824
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.6600
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.4312
Seen so far: 38464 samples
Training acc over epoch: 0.8862
Validation acc: 0.8869
Time taken: 2.88s

Start of epoch 1
Training loss (for one batch) at step 0: 0.2939
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.4878
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.2740
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.2621
Seen so far: 38464 samples
Training acc over epoch: 0.9011
Validation acc: 0.9045
Time taken: 1.66s
